<a href="https://colab.research.google.com/github/LiamDuero03/DS-Society-Project/blob/main/3-Data-Wrangling/Data-Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Data Wrangling & Quality Control

In this session, we will explore the essential steps of data cleaning. High-quality analysis depends on clean data, but real-world datasets are often "noisy."

We are working with a **synthetic weather dataset** specifically designed to showcase common data integrity issues. Your goal is to identify and resolve these problems using `pandas`.

### Key Issues to Address:
* **Outliers**: Physically impossible temperature readings that skew statistical averages.
* **Null Values**: Missing data points that can break analytical functions.
* **Naming Inconsistencies**: Variations in spelling, casing, and spacing for categorical data (Country and Weather Condition).



In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Replace this with your actual GitHub RAW URL
url = 'https://media.githubusercontent.com/media/LiamDuero03/DS-Society-Project/refs/heads/main/3-Data-Wrangling/messy_weather_data.csv'
df = pd.read_csv(url)

print("Dataset Loaded Successfully!")
df.head(10)

Dataset Loaded Successfully!


,Country,Temperature,Weather_Condition
0,Mexico,15.0,Rainy
1,USA,10.1,Cloudy
2,Mexico,30.4,Rainy
3,Mexico,27.7,NaN
4,Canadâ,28.2,Rainy
5,USA,29.3,sunny
6,Mexico,-999.0,SNOW
7,Canada,19.0,Rainy
8,Mexico,12.9,Snowy
9,mexico,31.6,Sunny


## 3.1 Exploratory Data Analysis (EDA): Identifying the Issues

Before we write any cleaning code, we must perform **Exploratory Data Analysis**. This phase allows us to diagnose the "health" of our dataset.

In the cell below, we use three primary methods to find our issues:
1.  **`.isnull()`**: To locate missing data points.
2.  **`.describe()`**: To find outliers by looking at the minimum and maximum values (checking for impossible temperatures).
3.  **`.unique()`**: To uncover naming inconsistencies caused by typos or formatting errors.

In [12]:
# Check for Nulls
print("--- Missing Values ---")
print(df.isnull().sum())

# Check for Outliers (Look at min and max!)
print("\n--- Numerical Summary ---")
print(df.describe())

# Check for Naming Inconsistencies
print("\n--- Unique Countries ---")
print(df['Country'].unique())

--- Missing Values ---
Country               0
Temperature          10
Weather_Condition    10
dtype: int64

--- Numerical Summary ---
       Temperature
count    90.000000
mean     26.552222
std     158.285726
min    -999.000000
25%      15.750000
50%      22.000000
75%      28.725000
max     999.900000

--- Unique Countries ---
['Mexico' 'USA' 'Canadâ' 'Canada' 'mexico' ' CANADA ' ' USA ' 'MEXICO '
 'uSA']


## 3.2 Resolving Naming Inconsistencies

Categorical columns like `Country` and `Weather_Condition` often suffer from "data siloing" where the same value is recorded in different formats.

To fix this, we apply a multi-step standardization process:
* **`.str.strip()`**: Removes accidental leading or trailing spaces.
* **`.str.upper()` / `.str.capitalize()`**: Forces uniform casing so "usa" and "USA" are treated as the same entity.
* **`.replace()`**: Manually maps misspellings or special characters (like "Canadâ") to the correct format.

In [13]:
# Clean Country names
df['Country'] = df['Country'].str.strip().str.upper()
df['Country'] = df['Country'].replace({'CANADÂ': 'CANADA', 'MEXICO': 'MEXICO'}) # Standardizing

# Clean Weather Condition
df['Weather_Condition'] = df['Weather_Condition'].str.strip().str.capitalize()
df['Weather_Condition'] = df['Weather_Condition'].replace({'Snow': 'Snowy'})

print("Fixed Naming Inconsistencies!")

Fixed Naming Inconsistencies!


## 3.3 Removing Numerical Outliers

Numerical outliers can severely skew your mean and standard deviation, leading to misleading analysis. In weather data, we can use **domain knowledge** to identify "impossible" values.

Since temperatures like **999°C** or **-999°C** are physically impossible on Earth, we create a logical filter to keep only the data that falls within a realistic range. This ensures our statistical model remains accurate and isn't pulled toward extreme sensor errors.

In [14]:
# We define a 'logical' range for weather on Earth
valid_range = (df['Temperature'] > -60) & (df['Temperature'] < 60)
df = df[valid_range]

print(f"Removed outliers. New Max Temp: {df['Temperature'].max()}")

Removed outliers. New Max Temp: 34.3


## 3.4 Handling Missing Values (Imputation & Removal)

Missing data (NaNs) can be handled in several ways depending on the importance of the column and the type of data it holds:

1.  **Grouped Imputation**: For `Temperature`, rather than using a global average, we use a more precise approach by filling missing values with the **median temperature of that specific country**. This preserves the geographic characteristics of the data.
2.  **Targeted Removal**: For `Weather_Condition`, since we cannot accurately guess if it was "Sunny" or "Rainy" without external data, we choose to **drop** only the rows where this specific column is null. This ensures we aren't introducing false categorical information into our dataset.

In [15]:
# --- STEP 1: Fill Temperature based on Country Median ---
# transform('median') keeps the index the same so it fits back into the dataframe perfectly
df['Temperature'] = df['Temperature'].fillna(
    df.groupby('Country')['Temperature'].transform('median')
)

# --- STEP 2: Remove rows where Weather_Condition is Null ---
# We use the 'subset' parameter to ONLY target rows missing weather data
df_clean = df.dropna(subset=['Weather_Condition']).copy()

print("1. Temperatures filled using Country-specific medians.")
print("2. Rows with missing Weather_Condition have been removed.")

1. Temperatures filled using Country-specific medians.
2. Rows with missing Weather_Condition have been removed.


## Final Verification

The final step in any data wrangling workflow is **verification**. We rerun our summary statistics to ensure:
* The **Temperature** range is now realistic (no more 999° values).
* The **Unique Values** for Country and Weather are standardized and clean.
* The **Data Quality** is consistent across the first few rows of the dataframe.

By comparing these results to our initial EDA, we can see exactly how the "noise" has been filtered out, leaving us with a reliable dataset for analysis.

In [16]:
print("--- Final Cleaned Data Statistics ---")
print(df_clean.describe())
print("\n--- Final Unique Values ---")
print(df_clean['Country'].unique())
print(df_clean['Weather_Condition'].unique())

# Show the first few rows of the beautiful, clean data
df_clean.head()

--- Final Cleaned Data Statistics ---
       Temperature
count    78.000000
mean     21.520513
std       7.189375
min      10.100000
25%      15.750000
50%      20.700000
75%      28.200000
max      34.300000

--- Final Unique Values ---
['MEXICO' 'USA' 'CANADA']
['Rainy' 'Cloudy' 'Sunny' 'Snowy']


,Country,Temperature,Weather_Condition
0,MEXICO,15.0,Rainy
1,USA,10.1,Cloudy
2,MEXICO,30.4,Rainy
4,CANADA,28.2,Rainy
5,USA,29.3,Sunny
